In [ ]:
import pandas as pd
import numpy as np
import sys

In [ ]:
sys.path.insert(0,"../../python/")
import rg17.text_cleaning as tc

# Load Parameters

In [ ]:
import sys
from datawand.parametrization import ParamHelper
ph = ParamHelper("../../", "TrendApproximation", sys.argv)

In [ ]:
unstemmed_file_path = ph.get("unstemmed_tweet_file_path")
stemmed_file_path = ph.get("stemmed_tweet_file_path")
stop_words_file_path = ph.get("stop_words_file_path")
player_file = ph.get("player_screen_names_file_path")
player_names_file_path = ph.get("player_names_file_path")

# 0. Load name parts and accounts

In [ ]:
player_accounts = []
with open(player_file) as f:
    for line in f:
        player_accounts += ["@%s" % p for p in line.rstrip()[2:-2].split('", "')]

In [ ]:
len(player_accounts)

In [ ]:
name_parts = []
with open(player_names_file_path) as f:
    for line in f:
        name_parts.append(line.rstrip())

In [ ]:
len(name_parts)

# 1. Cleaning english Tweet text

In [ ]:
from nltk.corpus import stopwords

In [ ]:
stopWords = set(stopwords.words('english'))

### Exclude "won" manually from nltk stop word set!!! 

In [ ]:
stopWords.remove("won")

### Add "amp" due to '&amp' removal

In [ ]:
stopWords.add("amp")

In [ ]:
len(stopWords)

## i.) Collect the list of fixed words

In [ ]:
list_of_fixed_words = [] 

## ii.) Add name parts to fixed words

In [ ]:
list_of_fixed_words += name_parts

## iii.) Cleaning examples

In [ ]:
tc.process_tweet_text("Tennis is my favourite sport in #Paris. Somebody is shouting!", stopWords, verbose=True)

In [ ]:
tc.process_tweet_text("win winning wins won winner champion victory", stopWords, verbose=True)

In [ ]:
tc.process_tweet_text("loser defeated lose lost beaten", stopWords)

In [ ]:
tc.lemmatize_words(["dogs","goods","win","won", "winner","run","ran","feet","are","is"])

In [ ]:
tc.stem_words(["dogs","goods","win","won", "winner","run","ran","feet","are","is"])

## iv.) cleaning Tweet text

In [ ]:
tweets_with_text = pd.read_csv(unstemmed_file_path, sep="|")

In [ ]:
list_of_fixed_words = [w.lower() for w in list_of_fixed_words]

try:
    for idx, row in tweets_with_text.iterrows():
        t_origi = row["text"]
        t_cleaned = tc.process_tweet_text(t_origi, stopWords)
except:
    print(t_origi)
    raise

In [ ]:
tweets_with_text["text"] = tweets_with_text["text"].apply(lambda x: tc.process_tweet_text(x, stopWords, list_of_fixed_words))

In [ ]:
tweets_with_text.to_csv(stemmed_file_path, sep="|", index=False)

# 2. Reloading cleaned file

In [ ]:
tweets_with_text = pd.read_csv(stemmed_file_path, sep="|")

In [ ]:
len(tweets_with_text)

### Exclude records after 2017-06-15

In [ ]:
tweets_with_text = tweets_with_text[tweets_with_text["time"] < 1497571200]
len(tweets_with_text)

# 3. Calculate TF-IDF for daily tweets

In [ ]:
tweets_with_text["date"] = pd.to_datetime(tweets_with_text["time"],unit="s")

In [ ]:
tweets_with_text.head()

In [ ]:
tweets_with_text["date_id"] = tweets_with_text["date"].apply(lambda x: str(x)[:-6])

In [ ]:
tweets_with_text["date_id"].value_counts()[-5:]

## i.) Concatenate tweet messages related to the same day

### Modify text before groupby: it is needed for the concatenation of messages

In [ ]:
tweets_with_text["text"] = tweets_with_text["text"].apply(lambda x: " " + str(x))

In [ ]:
grouped_docs = tweets_with_text.groupby("date_id")["text"].apply(lambda x: x.sum()).reset_index()

In [ ]:
grouped_docs = grouped_docs.drop(0,axis=0)

## ii.) Cleaning text with word size

In [ ]:
grouped_docs["words"] = grouped_docs["text"].apply(tc.tokenization_and_lowercase)
grouped_docs["num_words"] = grouped_docs["words"].apply(len)

In [ ]:
grouped_docs.head()

In [ ]:
all_words = set([])
for idx, row in grouped_docs.iterrows():
    all_words = all_words.union(set(row["words"]))

In [ ]:
len(all_words)

## Generate tweet text representations

In [ ]:
from collections import Counter

word_representations = []
for idx, row in grouped_docs.iterrows():
    cnt = Counter(row["words"])
    cnt_repr = dict(zip(all_words,np.zeros(len(all_words))))
    cnt_repr.update(dict(cnt))
    word_representations.append(cnt_repr)

In [ ]:
word_counts = pd.DataFrame(word_representations)

In [ ]:
word_counts_arr = word_counts.as_matrix()

In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer
transformer = TfidfTransformer(smooth_idf=False)

In [ ]:
tfidf_arr = transformer.fit_transform(word_counts_arr)

In [ ]:
tf_ids_df = pd.DataFrame(tfidf_arr.todense(), columns=list(word_counts.columns))

In [ ]:
tf_ids_df = tf_ids_df.replace(to_replace=0, value=np.nan)

tf_ids_df.head()

In [ ]:
sum_tf_idf = tf_ids_df.sum(axis=0)

In [ ]:
sum_tf_idf = sum_tf_idf.sort_values(ascending=False)

In [ ]:
player_tf_idfs = sum_tf_idf[sum_tf_idf.index.isin(player_accounts)]
non_player_tf_idfs = sum_tf_idf[~sum_tf_idf.index.isin(player_accounts)]

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

plt.subplots(figsize=(20,5))
plt.subplot(1,2,1)
sum_tf_idf.hist(bins=100)
plt.subplot(1,2,2)
player_tf_idfs.hist(bins=100)
plt.show()

In [ ]:
for i in range(100):
    print(non_player_tf_idfs.index[i],non_player_tf_idfs.ix[i])

# Make final list of words + Export

In [ ]:
def export_words(top):
    top_words = list(non_player_tf_idfs.head(top).index)
    player_related = player_accounts + name_parts
    player_words = list(np.setdiff1d(player_related,top_words))
    all_words = top_words + player_words 
    print(len(all_words))
    with open("/mnt/idms/fberes/network/roland_garros/data/rg17_%i_important_en_words_plus_players.txt" % top, 'w') as f_out:
        for w in all_words:
            f_out.write("%s\n" % w)

In [ ]:
for top in [3000,5000,8000]:
    export_words(top)